In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,MinMaxScaler
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
# from keras.utils import plot_model
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler

In [19]:
data = pd.read_excel('SPY500_stock_price_multiple_variables_1_10.xlsx', sheet_name='Data')
# Split data into features and target variable
X = data[['Date_Num','GDP','UNRATE','CPIAUCSL','FEDFUNDS']]
y = data['Close']

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Define the neural network model
model = Sequential()
model.add(Dense(50, activation='relu', input_dim=X.shape[1]))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()
# plot_model(model, to_file='model.png', show_shapes=True)
# Train the model
model.fit(X, y, epochs=400, validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 50)                300       
                                                                 
 dense_13 (Dense)            (None, 25)                1275      
                                                                 
 dense_14 (Dense)            (None, 10)                260       
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,846
Trainable params: 1,846
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
40/40 [==============================] - 1s 4ms/step - loss: 16278.9414 - val_loss: 111690.4922
Epoch 2/400
40/40 [==============================] - 0s 2ms/step - loss: 154

In [20]:
reference_date = dt.datetime(1993, 1, 25)  # Set the reference date to Jan 25, 1993
Date_r = (dt.datetime(2023, 1, 1) - reference_date).days  # Calculate the number of days since the reference date
# Make predictions on new data
new_data = pd.DataFrame({'Date_Num': [Date_r],
                      'GDP': [20187], 
                      'UNRATE': [3.5],
                      'CPIAUCSL': [6.44], 
                      'FEDFUNDS': [4.10]})
new_data_scaled = scaler.transform(new_data)
prediction = model.predict(new_data_scaled)
print(prediction)


1/1 [==============================] - 0s 38ms/step
[[378.5505]]


In [21]:
# Export our model to HDF5 file
model.save("trained_spy_prediction_06.h5")

In [22]:
# Import the model to a new object
saved_model = tf.keras.models.load_model('trained_spy_prediction_06.h5')

In [23]:
predicted_data = pd.read_excel('SPY500_stock_price_multiple_variables_1_10.xlsx', sheet_name='EI_Prediction')

In [24]:
reference_date = dt.datetime(1993, 1, 25)  # Set the reference date to Jan 25, 1993
prediction_list = []

for _, row in predicted_data.iterrows():
    Date_r = (row['Date'] - reference_date).days  # Calculate the number of days since the reference date
    # Make predictions on new data
    new_data = pd.DataFrame({'Date_Num': [Date_r],
                        'GDP': [row.GDP_Est], 
                        'UNRATE': [row.UR_Pred],
                        'CPIAUCSL': [row.CPI_Est], 
                        'FEDFUNDS': [row.IR_Est]})
    new_data_scaled = scaler.transform(new_data)
    prediction = saved_model.predict(new_data_scaled)
    prediction_list.append(float(prediction[0][0]))

1/1 [==============================] - 0s 14ms/step


In [25]:
predicted_data['Prediction'] = prediction_list

In [26]:
predicted_data

,Date,Date_Text,Month,Day,Year,Quarter,YearMonth,GDP_Est,CPI_Est,IR_Est,UR_Pred,Prediction
0,2023-01-02,01/02/2023,1,2,2023,2023Q1,2023M1,20187.495000,8.05,5.2,3.865880,394.453979
1,2023-01-09,01/09/2023,1,9,2023,2023Q1,2023M1,20217.458558,8.05,5.2,3.865880,394.980133
2,2023-01-16,01/16/2023,1,16,2023,2023Q1,2023M1,20247.422115,8.05,5.2,3.865880,395.506378
3,2023-01-23,01/23/2023,1,23,2023,2023Q1,2023M1,20277.385673,8.05,5.2,3.865880,396.032501
4,2023-01-30,01/30/2023,1,30,2023,2023Q1,2023M1,20307.349231,8.05,5.2,3.865880,396.558685
...,...,...,...,...,...,...,...,...,...,...,...,...
100,2024-12-02,12/02/2024,12,2,2024,2024Q4,2024M12,22063.129231,2.59,4.1,4.712981,360.446808
101,2024-12-09,12/09/2024,12,9,2024,2024Q4,2024M12,22069.744423,2.59,4.1,4.712981,360.733307
102,2024-12-16,12/16/2024,12,16,2024,2024Q4,2024M12,22076.359615,2.59,4.1,4.712981,361.019836
103,2024-12-23,12/23/2024,12,23,2024,2024Q4,2024M12,22082.974808,2.59,4.1,4.712981,361.306305


In [27]:
predicted_data.to_excel("predicted_data.xlsx", sheet_name = 'Sheet1', index=True)